# UNSW-NB15: Preparação da Data

## Carregamento das bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Carregamento dos datasets
train = pd.read_csv('./train_alldata_preparacao.csv')
test = pd.read_csv('./test_alldata_preparacao.csv')

In [3]:
# Retirando a coluna 'label' que não terá relevancia nesta parte do problema

train = train.drop(columns='label')
test = test.drop(columns='label')

print(train.columns)
print()
print(test.columns)
print()

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'dload', 'dloss',
       ...
       'state_CLO', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT',
       'state_PAR', 'state_REQ', 'state_RST', 'state_URN', 'state_no'],
      dtype='object', length=178)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'dload', 'dloss',
       ...
       'state_CLO', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT',
       'state_PAR', 'state_REQ', 'state_RST', 'state_URN', 'state_no'],
      dtype='object', length=178)



In [4]:
# Funções uteis
def multi_corr(col1, col2="attack_cat", df=train):
    '''
    CORRELAÇAO + LOG CORRELAÇAO
    A função calcula o coeficiente de correlação de Pearson entre col1 e col2 usando o método corr() e guarda o valor numa variável chamada corr.
    A função então aplica a função np.log1p aos valores em col1.
        A função np.log1p() é usada para calcular o logaritmo natural [1 + col1].
        [A transformação de log pode ser útil para reduzir
        o impacto de outliers ou valores extremos nos dados
        e pode tornar a relação entre as duas variáveis mais
        linear ou mais fácil de modelar.] 
    Calcula a correlação entre col1 transformada e col2 e guarda o valor numa variável chamada log_corr.
    Por fim, a função imprime os valores de corr e log_corr.
    '''

    if df[col2].dtype == 'object':
        dummy_df = pd.get_dummies(df[col2])
        corr = df[col1].corr(dummy_df.iloc[:, 0])
        log_corr = df[col1].apply(np.log1p).corr(dummy_df.iloc[:, 0])
    else:
        corr = df[col1].corr(df[col2])
        log_corr = df[col1].apply(np.log1p).corr(df[col2])

    print("Correlation: {}\nlog_Correlation: {}".format(corr, log_corr))

    if log_corr > corr:
        return col1
    else:
        return None

In [5]:
def corr(col1, col2="attack_cat", df=train):
    """
    CORRELAÇAO
    A função calcula o coeficiente de correlação de Pearson entre col1 e col2 e guarda o valor numa variável chamada corr.
    Devolve o valor de corr.
    """
    return df[[col1, col2]].corr().iloc[0,1]

## Remoção das colunas com correlações elevadas

In [6]:
# Seleção das colunas com correlações elevadas relativemante a outras
# Referência: https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
corr_matrix = train.corr().abs()

# Selecionamos apenas o triangulo superior da matrix de correlação
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Encontra o index das colunas com uma correlação maior que 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [7]:
# Não queremos usar estas colunas nos gráficos porque têm alta corr
# E provavelmente têm o mesmo tipo de comportamento que uma coluna já traçada num gráfico
print(to_drop)

['dbytes', 'dloss', 'state_FIN']


In [8]:
# Irá ser mantida a coluna 'state_FIN', pois o seu comportamento está relacionado com as colunas 'state' e será util na construção dos classificadores

to_drop.remove('state_FIN')
print(to_drop)

['dbytes', 'dloss']


In [9]:
# Irão ser removidas as colunas do dataset de treino e teste

train.drop(columns=to_drop, inplace=True)
test.drop(columns=to_drop, inplace=True)

In [10]:
train.shape, test.shape

((228664, 176), (80650, 176))

## Aplicar log1p nas Colunas numéricas

        A função np.log1p() é usada para calcular o log [1 + col1].
        [A transformação de log pode ser útil para reduzir
        o impacto de outliers ou valores extremos nos dados
        e pode tornar a relação entre as duas variáveis mais
        linear ou mais fácil de modelar.] 

Durante  Parte 2, descobrimos que poucas colunas numéricas mostram uma melhor visualização para curvas PDF((Probability Density Function)) se aplicarmos log1p às colunas.

Então, pensei em tentar log1p em todas as colunas e verificar o valor de correlação da coluna original e da coluna log1p com a coluna de alvo ( "attack_cat")

In [11]:
# Obtendo o número de valores únicos de todas as colunas
# Se os valores únicos forem altos, isso significa que tem um conjunto contínuo de valores (indicador de coluna numérica)
col_unique_values = train.nunique()

In [12]:
# Se os valores unicos forem maiores que um determinado limite, iremos verificar a sua correção
col = col_unique_values[col_unique_values>200].index
print(col)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'rate', 'dload', 'sinpkt', 'dinpkt',
       'djit', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean'],
      dtype='object')


In [13]:
columns_with_better_log_corr = []
for column in col:
    print("{:-^30}".format(column))
    result = multi_corr(column)
    if result is not None:
        columns_with_better_log_corr.append(result)

log1p_col = columns_with_better_log_corr

print("Colunas com melhor correlação log: {}".format(columns_with_better_log_corr))

-------------dur--------------
Correlation: 0.05100108703853802
log_Correlation: 0.005788176821005978
------------spkts-------------
Correlation: -0.0014144004973209439
log_Correlation: -0.11209067632161743
------------dpkts-------------
Correlation: -0.011906366494527334
log_Correlation: -0.1676412388427909
------------sbytes------------
Correlation: 0.01472733023367995
log_Correlation: -0.07006802973761533
-------------rate-------------
Correlation: 0.12271841422533296
log_Correlation: 0.12461466896005878
------------dload-------------
Correlation: -0.08234303735509164
log_Correlation: -0.20182468341993404
------------sinpkt------------
Correlation: -0.03667968365447619
log_Correlation: -0.06947143055653465
------------dinpkt------------
Correlation: -0.01624966702851681
log_Correlation: -0.09123212307771117
-------------djit-------------
Correlation: -0.030375350583872446
log_Correlation: -0.12261779350570452
------------tcprtt------------
Correlation: -0.04949787075425228
log_Corre

In [14]:
def log1p_transform(col, df=train):
    '''
    Esta função aplica a função np.log1p() a uma determinada coluna de um dataframe. 
    De seguida, cria uma nova coluna com o nome da coluna original anexado com _log1p e guarda os valores transformados em log nessa nova coluna.
    Por fim, remove a coluna original do DataFrame.
    '''
    new_col = col+'_log1p'
    df[new_col] = df[col].apply(np.log1p)
    df.drop(col, axis=1, inplace=True)

In [15]:
# Transformando as colunas numéricas com log1p nos conjuntos treino e teste
for col in log1p_col:
    log1p_transform(col, df=train)
    log1p_transform(col, df=test)

print(train.shape, test.shape)
print()
print(train.columns)
print()
print(test.columns)

(228664, 176) (80650, 176)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'sttl', 'dttl', 'dload', 'sinpkt',
       'dinpkt', 'djit',
       ...
       'state_ECO', 'state_FIN', 'state_INT', 'state_PAR', 'state_REQ',
       'state_RST', 'state_URN', 'state_no', 'rate_log1p', 'smean_log1p'],
      dtype='object', length=176)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'sttl', 'dttl', 'dload', 'sinpkt',
       'dinpkt', 'djit',
       ...
       'state_ECO', 'state_FIN', 'state_INT', 'state_PAR', 'state_REQ',
       'state_RST', 'state_URN', 'state_no', 'rate_log1p', 'smean_log1p'],
      dtype='object', length=176)


In [16]:
# Separação dos conjuntos de dados de treino e teste em variáveis ​​de recursos (x_train, x_test) e de destino (y_train, y_test).
# A função drop é usada para remover a coluna de rótulo/alvo dos conjuntos de dados e armazená-los em y_train e y_test, enquanto as colunas restantes são armazenadas em x_train e x_test.

x_train, y_train = train.drop(columns=['attack_cat']), train['attack_cat']
x_test, y_test = test.drop(columns=['attack_cat']), test['attack_cat']

In [17]:
print(x_train.shape, y_train.shape)
print()
print(x_test.shape, y_test.shape)

(228664, 175) (228664,)

(80650, 175) (80650,)


In [18]:
x_train.head()

dur  spkts  dpkts  sbytes  sttl  dttl          dload      sinpkt  \
0  0.707822     10      8     534   254   252    3503.705566   78.545556   
1  0.001153      2      2     132    31    29  568950.562500    0.009000   
2  0.000008      2      0     200   254     0       0.000000    0.008000   
3  0.000004      2      0     672   254     0       0.000000    0.004000   
4  1.611537     10      8     930   254   252    1538.903564  176.559444   

       dinpkt        djit  ...  state_ECO  state_FIN  state_INT  state_PAR  \
0   90.806430  137.357734  ...        0.0        1.0        0.0        0.0   
1    0.007000    0.000000  ...        0.0        0.0        0.0        0.0   
2    0.000000    0.000000  ...        0.0        0.0        1.0        0.0   
3    0.000000    0.000000  ...        0.0        0.0        1.0        0.0   
4  221.066422  431.869125  ...        0.0        1.0        0.0        0.0   

   state_REQ  state_RST  state_URN  state_no  rate_log1p  smean_log1p  
0        0.0        0.0        0.0       0.0    3.219569     3.988984  
1        0.0        0.0        0.0       0.0    7.864385     4.204693  
2        0.0        0.0        0.0       0.0   11.736077     4.615121  
3        0.0        0.0        0.0       0.0   12.429220     5.820083  
4        0.0        0.0        0.0       0.0    2.446593     4.543295  

[5 rows x 175 columns]

## Normalização Z-Score / Standardizing
    
Como vimos, o intervalo de valores de algumas colunas neste conjunto de dados é muito grande. <br>
Assim sendo, vamos colocar tudo num determinado intervalo usando standardscaler. Depois deste processo, todos os recursos terão média 0 e um desvio padrão de 1

In [19]:
col_unique_values = x_train.nunique()
num_col = col_unique_values[col_unique_values>200].index
print(num_col)

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dload', 'sinpkt', 'dinpkt', 'djit',
       'tcprtt', 'synack', 'ackdat', 'dmean', 'rate_log1p', 'smean_log1p'],
      dtype='object')


In [20]:
# Aplicar a normalização
scaler = StandardScaler()
scaler = scaler.fit(x_train[num_col])   # ajustar o scaler às colunas numéricas (col) de x_train

x_train[num_col] = scaler.transform(x_train[num_col])   # transforma as colunas numéricas de x_train usando o scaler
x_test[num_col] = scaler.transform(x_test[num_col])   # transforma as colunas numéricas de x_train usando o scaler

x_train.head()

dur     spkts     dpkts    sbytes  sttl  dttl     dload    sinpkt  \
0 -0.092420 -0.107414 -0.146376 -0.047428   254   252 -0.349790 -0.155085   
1 -0.208483 -0.172228 -0.200467 -0.050053    31    29 -0.158164 -0.163881   
2 -0.208671 -0.172228 -0.218497 -0.049609   254     0 -0.350978 -0.163881   
3 -0.208672 -0.172228 -0.218497 -0.046527   254     0 -0.350978 -0.163882   
4  0.056007 -0.107414 -0.146376 -0.044842   254   252 -0.350456 -0.144108   

     dinpkt      djit  ...  state_ECO  state_FIN  state_INT  state_PAR  \
0 -0.007338 -0.122013  ...        0.0        1.0        0.0        0.0   
1 -0.083552 -0.151626  ...        0.0        0.0        0.0        0.0   
2 -0.083558 -0.151626  ...        0.0        0.0        1.0        0.0   
3 -0.083558 -0.151626  ...        0.0        0.0        1.0        0.0   
4  0.101997 -0.058520  ...        0.0        1.0        0.0        0.0   

   state_REQ  state_RST  state_URN  state_no  rate_log1p  smean_log1p  
0        0.0        0.0        0.0       0.0   -1.060250    -0.743290  
1        0.0        0.0        0.0       0.0    0.119089    -0.444800  
2        0.0        0.0        0.0       0.0    1.102128     0.123134  
3        0.0        0.0        0.0       0.0    1.278120     1.790514  
4        0.0        0.0        0.0       0.0   -1.256511     0.023744  

[5 rows x 175 columns]

In [21]:
file_path = 'dataPronta/'

In [22]:
# Verificar se as colunas de treino e teste são as mesmas
all(x_train.columns == x_test.columns)

True

In [23]:
# Guardar todos os dados para usar no treino e teste das redes neuronais
pickle.dump((x_train, y_train), open('./dataPronta/final_train.pkl', 'wb'))
pickle.dump((x_test, y_test), open('./dataPronta/final_test.pkl', 'wb'))

In [24]:
x_test.shape, y_test.shape, x_train.shape, y_train.shape

((80650, 175), (80650,), (228664, 175), (228664,))